### Aujourd'hui, nous allons scrapper Producthunt avec Selenium

Selenium est un outil crée initialement pour automatiser des test sur des sites web. Il est donc bien utile lorsque des informations sont accesibles en cliquant sur des liens. Un boutton par exemple est un élement sur lequel il tres diffcile d'obtenir le lien. Beautifull soup devient alors limité.
Dans ce cas, utilisez Selenium

### Charger les librairies

In [1]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os
date=strftime("%Y-%m-%d")


### Installer Selenium en suivant cette notice

https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium/bin

nb: Linux : mettez votre geckodriver (l'extension telechargé) dans le chemin equivalent chez vous à /home/david/.local/bin

Nous allons simuler une recherche sur le site officiel de Phython

In [2]:
import selenium

# Le module selenium.webdriver fournit toutes les implémentations de WebDriver. Les implémentations WebDriver 
# actuellement prises en charge sont Firefox, Chrome, IE et Remote. La classe Keys fournit des touches dans 
# le clavier telles que RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# Ensuite, l'instance de Firefox WebDriver est créée.
driver = webdriver.Firefox()
# La méthode driver.get mènera à une page donnée par l'URL. WebDriver attendra que la page soit complètement 
# chargée (c'est-à-dire que l'événement “onload” s'est déclenché) avant de rendre le contrôle à votre script. 
# Il est à noter que si votre page utilise beaucoup d'AJAX au chargement, WebDriver peut ne pas savoir 
# quand il s'est complètement chargé:
driver.get("http://www.python.org")
# La ligne suivante est une affirmation confirmant que le titre contient le mot «Python»
assert "Python" in driver.title
# WebDriver propose plusieurs méthodes pour rechercher des éléments à l'aide de l'une des méthodes 
# find_element_by_ * . Par exemple, l'élément de texte d'entrée peut être localisé par son attribut name en 
# utilisant la méthode find_element_by_name. Il existe plusieurs autres methodes pour rechercher sa balise.
elem = driver.find_element_by_name("q")
# Ensuite, nous envoyons des clés. C’est similaire à la saisie de touches à l’aide de votre clavier. 
# Des clés spéciales peuvent être envoyées à l'aide de la classe Keys importée de selenium.webdriver.common.keys . 
# Pour des raisons de sécurité, nous effacerons tout texte pré-rempli dans le champ de saisie 
# (par exemple, «Rechercher») afin que cela n'affecte pas nos résultats de recherche:


elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)

#Après soumission de la page, vous devriez obtenir le résultat s'il y en a. Pour vous assurer que certains résultats 
# sont trouvés, faites une assertion:
assert "No results found." not in driver.page_source
driver.close()

#### Ouvre le code source de cette page (HTML) et verifier que la zone de recherche (le champ) s'appelle bien "q"

input id="id-search-field" name="q" type="search" role="textbox" class="search-field placeholder" placeholder="Search" value="" tabindex="1"

### Recuperer un numéro de telephone sur une annonce du bon coin

#### Exemple

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

url='https://www.leboncoin.fr/sports_hobbies/1536839557.htm/'

driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(url)


python_button = driver.find_elements_by_xpath('//div[@data-reactid="307"]')[0]
python_button.click()

# Et après, c'est comme avec Beautiful soup
soup=BeautifulSoup(driver.page_source)

driver.close()

# Et après, c'est comme avec Beautiful soup
for elem in soup.find_all('a',attrs={"data-qa-id" :"adview_number_phone_contact" }):
    print(elem.text)

IndexError: list index out of range

### En partant d'une annonce du bon coin, recuperer tous les éléments dicponibles permettant de bien définir le produit mis en vente. Utilisez selenium pour le numéro de telephone

### API (Application Program Interface)

Enesemble d'outils et de méthodes qui autorisent différentes applications àinteragir entre elles. Dans le cas d'un web service, nous pouvons y récupérer de la data dynamiquement. En utilisant correctemen une API, nous pouvons ainsi otenire en temps réel, les modifications apportées sur un site "mère".

Par exemple, nous allons récupérer des news en ligne, par exemple celles du site "L'equipe"

Suivez les directives de https://newsapi.org/s/lequipe-api pour récupérer une cle de connexion "API key"

Your API key is: 73bbb95f8ecb49b499113a46481b4af1


nb: Il est fréquent qu'une clé ne fonctionne au bout de quelques temps 5mins 30 mins 1 j ...
Donc ne sautez pas en l'air s'il on vous renvoi un message d'erreur.

In [101]:
import requests
key='73bbb95f8ecb49b499113a46481b4af1'
url ='https://newsapi.org/v2/top-headlines?sources=lequipe&apiKey='+key
response = requests.get(url)

# Ici le format de réponse est un fichier json, il s'utilise comme un dictionnaire

print (response.json())

{'status': 'ok', 'totalResults': 10, 'articles': [{'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Real Madrid : Gareth Bale met fin à sa disette d&#039;une belle reprise de volée face à Huesca (vidéo)', 'description': "L'attaquant du Real Madrid Gareth Bale a mis fin ce dimance à sa disette en Liga en ouvrant le score face à Huesca d'une belle reprise.", 'url': 'https://www.lequipe.fr/Football/Actualites/Real-madrid-gareth-bale-met-fin-a-sa-disette-d-une-belle-reprise-de-volee-face-a-huesca-video/966796', 'urlToImage': 'https://medias.lequipe.fr/img-photo-jpg/-oscar-j-barroso-oscar-j-barroso-afp7/1500000001061170/0:0,2000:1333-624-416-75/cb91a.jpg', 'publishedAt': '2018-12-09T16:02:00+00:00', 'content': "801 minutes. C'est le temps de jeu qui séparait le dernier but de Gareth Bale en Liga de celui qu'il a inscrit ce dimanche en ouvrant le score sur le terrain d'Huesca. Le Gallois a repris de belle manière un centre d'Odriozola pour mettre fin à sa dise

In [103]:
dic=response.json()
print(dic.keys())

dict_keys(['status', 'totalResults', 'articles'])


In [109]:
for elem in list(dic.keys()):
    print('##############################################')
    print("clé: ",elem,"// values: ", dic[elem])

##############################################
clé:  status // values:  ok
##############################################
clé:  totalResults // values:  10
##############################################
clé:  articles // values:  [{'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Real Madrid : Gareth Bale met fin à sa disette d&#039;une belle reprise de volée face à Huesca (vidéo)', 'description': "L'attaquant du Real Madrid Gareth Bale a mis fin ce dimance à sa disette en Liga en ouvrant le score face à Huesca d'une belle reprise.", 'url': 'https://www.lequipe.fr/Football/Actualites/Real-madrid-gareth-bale-met-fin-a-sa-disette-d-une-belle-reprise-de-volee-face-a-huesca-video/966796', 'urlToImage': 'https://medias.lequipe.fr/img-photo-jpg/-oscar-j-barroso-oscar-j-barroso-afp7/1500000001061170/0:0,2000:1333-624-416-75/cb91a.jpg', 'publishedAt': '2018-12-09T16:02:00+00:00', 'content': "801 minutes. C'est le temps de jeu qui séparait le dernier but de Garet

In [123]:
# Et là nous avons des listes dans des dictionnaires c'est du json mais c'est toujours la meme chose
# Nous allons decouvrir les informations de la clé articles
for elem in enumerate(dic['articles']):
    print('###############################################')
    print(elem)

###############################################
(0, {'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Real Madrid : Gareth Bale met fin à sa disette d&#039;une belle reprise de volée face à Huesca (vidéo)', 'description': "L'attaquant du Real Madrid Gareth Bale a mis fin ce dimance à sa disette en Liga en ouvrant le score face à Huesca d'une belle reprise.", 'url': 'https://www.lequipe.fr/Football/Actualites/Real-madrid-gareth-bale-met-fin-a-sa-disette-d-une-belle-reprise-de-volee-face-a-huesca-video/966796', 'urlToImage': 'https://medias.lequipe.fr/img-photo-jpg/-oscar-j-barroso-oscar-j-barroso-afp7/1500000001061170/0:0,2000:1333-624-416-75/cb91a.jpg', 'publishedAt': '2018-12-09T16:02:00+00:00', 'content': "801 minutes. C'est le temps de jeu qui séparait le dernier but de Gareth Bale en Liga de celui qu'il a inscrit ce dimanche en ouvrant le score sur le terrain d'Huesca. Le Gallois a repris de belle manière un centre d'Odriozola pour mettre fin à sa di

In [128]:
# Donc s'il on continue, ça nous donne un autre dictionnaire !
for elem in dic['articles'][0].keys():
    print(' Cle : ',elem,'Values : ',dic['articles'][0][elem])

 Cle :  source Values :  {'id': 'lequipe', 'name': "L'equipe"}
 Cle :  author Values :  L'EQUIPE
 Cle :  title Values :  Real Madrid : Gareth Bale met fin à sa disette d&#039;une belle reprise de volée face à Huesca (vidéo)
 Cle :  description Values :  L'attaquant du Real Madrid Gareth Bale a mis fin ce dimance à sa disette en Liga en ouvrant le score face à Huesca d'une belle reprise.
 Cle :  url Values :  https://www.lequipe.fr/Football/Actualites/Real-madrid-gareth-bale-met-fin-a-sa-disette-d-une-belle-reprise-de-volee-face-a-huesca-video/966796
 Cle :  urlToImage Values :  https://medias.lequipe.fr/img-photo-jpg/-oscar-j-barroso-oscar-j-barroso-afp7/1500000001061170/0:0,2000:1333-624-416-75/cb91a.jpg
 Cle :  publishedAt Values :  2018-12-09T16:02:00+00:00
 Cle :  content Values :  801 minutes. C'est le temps de jeu qui séparait le dernier but de Gareth Bale en Liga de celui qu'il a inscrit ce dimanche en ouvrant le score sur le terrain d'Huesca. Le Gallois a repris de belle manièr

### Faites un script qui permet de prendre les details des dix dernieres news de l'équipe ou d'un autre sites. Rangez les dans un beau fichier csv ou excel